### 문제 4-1 : 카페 메뉴 도구(Tool) 호출 체인 구현

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

sk
WD


In [2]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools import tool

def build_cafe_db():
    loader = TextLoader("../../data/cafe_menu_data.txt", encoding="utf-8")
    docs = loader.load()

    splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    split_docs = splitter.split_documents(docs)

    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    db = FAISS.from_documents(split_docs, embeddings)

    db.save_local("./db/cafe_db")
    print("벡터 DB 생성 완료: ./db/cafe_db")

if __name__ == "__main__":
    build_cafe_db()

c:\Users\USER\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-CGunLKE6-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


벡터 DB 생성 완료: ./db/cafe_db


In [3]:
from langchain.agents import tool
from langchain_community.tools import TavilySearchResults
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

tavily_search = TavilySearchResults(max_results=3)

@tool
def tavily_search_func(query: str) -> str:
    """웹에서 최신 정보를 검색합니다."""
    return tavily_search.run(query)

wiki = WikipediaAPIWrapper()

@tool
def wiki_summary(topic: str) -> str:
    """위키피디아에서 주제를 검색하고 요약합니다."""
    return wiki.run(topic)

@tool
def db_search_cafe_func(query: str):
    """카페 메뉴 벡터 DB에서 정보를 검색합니다."""
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    db = FAISS.load_local("./db/cafe_db", embeddings, allow_dangerous_deserialization=True)
    docs = db.similarity_search(query, k=2)
    return [doc.page_content for doc in docs]


C:\Users\USER\AppData\Local\Temp\ipykernel_1568\2004809752.py:7: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_search = TavilySearchResults(max_results=3)


In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

llm_with_tools = llm.bind_tools([tavily_search_func, wiki_summary, db_search_cafe_func])

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 카페 메뉴 정보를 제공하는 똑똑한 어시스턴트입니다."),
    ("human", "{question}")
])

def run_chain(question: str):
    chain = prompt | llm_with_tools
    result = chain.invoke({"question": question})
    
    print("질문:", question)
    print("모델 응답:", result.content)

    if hasattr(result, "tool_calls"):
        print("\n도구 호출:")
        for call in result.tool_calls:
            print(f"- {call['name']}({call['args']})")

    return result


In [6]:
if __name__ == "__main__":
    run_chain("아메리카노의 가격과 특징은 무엇인가요?")

질문: 아메리카노의 가격과 특징은 무엇인가요?
모델 응답: 

도구 호출:
- db_search_cafe_func({'query': '아메리카노'})
